In [1]:
import os, sys
import glob
import pickle
import numpy as np
import pandas as pd
import cclib

from openbabel import pybel

import rdkit
import rdkit.Chem as chem
import rdkit.Chem.AllChem as allchem

from mordred import ( 
    Calculator, 
    descriptors, 
    GeometricalIndex, # GeomRadius
    Weight, # Weight
    TopologicalIndex, #Radius
)

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def mfs(s):
    return chem.MolFromSmiles(s)

def mts(m):
    return chem.MolToCXSmiles(m)

def xyz2mol(xyzfile):
    mol = next(pybel.readfile('xyz', xyzfile))
    mol.write('mol', (xyzfile[:-4] + '.mol'), overwrite=True)

    
def write_xyz(atomnos, atomcoords, name, filename):
    with open(filename, 'w') as f:
        f.write(f'{name}\n\n')
        for atomno, atomcoord in zip(atomnos, atomcoords):
            atomsym = atom_mapping[atomno]
            f.write(f' {atomsym}\t{atomcoord[0]:.5f}  {atomcoord[1]:.5f}  {atomcoord[2]:.5f}\n')

In [3]:
# read in the results
# known constraint is that the entry exists in df_resutls
df = pd.read_csv('df_results.csv')
print(df.shape)
print(df.columns)
df.head()

(111, 10)
Index(['E_hull', 'mu_mu_B', 'm_star', 'm_e_star', 'm_h_star', 'E_f', 'bandgap',
       'molcat', 'metal', 'halogen'],
      dtype='object')


,E_hull,mu_mu_B,m_star,m_e_star,m_h_star,E_f,bandgap,molcat,metal,halogen
0,0.0,0.0,7.11,7.14,>1000,-0.565,2.54,H3S,In,Cl
1,0.0,20.0,44.46,>1000,44.46,-0.467,1.52,MP,Mn,Cl
2,0.0,0.0,0.34,0.35,6.64,-1.056,4.00,NH4,Ca,Br
3,0.0,0.0,0.52,1.09,0.99,-0.415,3.06,MP,Ge,Cl
4,0.0,0.0,363.95,>1000,417.06,-0.231,1.03,H3S,Te,Br


In [12]:
molcats = ['H3S', 'NH4', 'MS', 'MA', 'MP', 'FA', 'EA', 'G', 'AA', 'ED', 'tBA']
metals = ['Be', 'Mg', 'Ca', 'Sr', 'Ba', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Pd', 'Pt', 'Cu', 'Ag',
          'Au', 'Zn', 'Cd', 'Hg', 'Ga', 'In', 'Tl', 'Si', 'Ge', 'Sn', 'Pb', 'Bi', 'S', 'Se', 'Te']
halogens = ['F', 'Cl', 'Br', 'I']

# define a mapping between the names for the organic cations
molcats_mapping = {
    'sulfonium': 'H3S',
    'ammonium': 'NH4',
    'methylsulfonium': 'MS',
    'methylammonium': 'MA', 
    'methylphosphonium': 'MP',
    'formamidinium': 'FA',
    'ethylammonium': 'EA',
    'guanidinium': 'G',
    'acetamidinium': 'AA',
    'ethylenediamine': 'ED',
    'tert-butylammonium': 'tBA',
}

atom_mapping = {
    1: 'H',
    6: 'C',
    7: 'N',
    8: 'O',
    16: 'S',
    15: 'P',
}

# METALS DESCRIPTORS

desc_metals = {
    'electron_affinity': {
        'Be': -0.5, 'Mg': -0.4, 'Ca': 0.02455, 'Sr': 0.05206, 'Ba': 0.14462, 'Cr': 0.67584,
        'Mn': -0.5, 'Fe': 0.153236, 'Co': 0.66226, 'Ni': 1.15716, 'Pd': 0.56214,
        'Pt': 2.12510, 'Cu': 1.23578, 'Ag': 1.30447, 'Au': 2.308610, 'Zn': -0.6,
        'Cd': -0.7, 'Hg': -0.5, 'Ga': 0.30120, 'In': 0.38392, 'Tl': 0.320053,
        'Si': 1.389521, 'Ge': 1.2326764, 'Sn': 1.112070, 'Pb': 0.356721, 'Bi': 0.942362,
        'S': 2.0771042, 'Se': 2.0206047, 'Te': 1.970875,
    },
    'electronegativity': {
        'Be': 1.57, 'Mg': 1.31, 'Ca': 1.00, 'Sr': 0.95, 'Ba': 0.89,
        'Cr': 1.66, 'Mn': 1.55, 'Fe': 1.83, 'Co': 1.88, 'Ni': 1.91,
        'Pd': 2.20, 'Pt': 2.28, 'Cu': 1.90, 'Ag': 1.93, 'Au': 2.54,
        'Zn': 1.65, 'Cd': 1.69, 'Hg': 2.00, 'Ga': 1.81, 'In': 1.78,
        'Tl': 1.62, 'Si': 1.90, 'Ge': 2.01, 'Sn': 1.96, 'Pb': 2.33,
        'Bi': 2.02, 'S': 2.58, 'Se': 2.55, 'Te': 2.1,
    },
    'ionization_energy': {
        'Be': 9.32269, 'Mg': 7.64624, 'Ca': 6.11316, 'Sr': 5.6949, 'Ba': 5.21170, 'Cr': 6.7665,
        'Mn': 7.43402, 'Fe': 7.9024, 'Co': 7.8810, 'Ni': 7.6398, 'Pd': 8.3369, 'Pt': 8.9587,
        'Cu': 7.72638, 'Ag': 7.5762, 'Au': 9.2255, 'Zn': 9.3942, 'Cd': 8.9938, 'Hg': 10.43750,
        'Ga': 5.99930, 'In': 5.78636, 'Tl': 6.1082, 'Si': 8.15169, 'Ge': 7.8994, 'Sn': 7.3439,
        'Pb': 7.41666, 'Bi': 7.2856, 'S': 10.36001, 'Se': 9.75238, 'Te': 9.0096,
    },
    'total_mass': {
        'Be': 9.01218, 'Mg': 24.305, 'Ca': 40.08, 'Sr': 87.62, 'Ba': 137.33, 'Cr': 51.996,
        'Mn': 54.9380, 'Fe': 55.847, 'Co': 58.9332, 'Ni': 58.70, 'Pd': 106.4, 'Pt': 195.09,
        'Cu': 63.546, 'Ag': 107.868, 'Au': 196.9665, 'Zn': 65.38, 'Cd': 112.41, 'Hg': 200.59,
        'Ga': 69.72, 'In': 114.82, 'Tl': 204.37, 'Si': 28.0855, 'Ge': 72.59, 'Sn': 118.69,
        'Pb': 207.2, 'Bi': 208.9804, 'S': 32.06, 'Se': 78.96, 'Te': 127.60,
    }
}

# HALOGENS DESCRIPTORS

desc_halogens = {
    'electron_affinity': {
        'F': 3.4011898,
        'Cl': 3.612725,
        'Br': 3.363588,
        'I': 3.0590465,
    },
    'electronegativity': {
        'F': 3.98,
        'Cl': 3.16,
        'Br': 2.96,
        'I': 2.66,
    },
    'ionization_energy': {
        'F': 17.42282,
        'Cl': 12.96764,
        'Br': 11.81381,
        'I': 10.45126,
    },
    'total_mass': {
        'F': 18.998403,
        'Cl': 35.453,
        'Br': 79.904,
        'I': 126.9045,
    }
}


## Get descriptors for the organic cations 

Descritors considered: 
   
* homo_e [eV]
* lumo_e [eV]
* dipole moment norm [D]
* atomization energy [eV]
* scf energy [eV]
* radius of gyration [?]
* total mass [amu or g/mol]


In [28]:
# parse out the dft results
T = 298.15 # K
kb = 8.617333262e-5 # eV K^{-1}

def get_boltzmann_avg(prop, scf_e):
    prop = np.array(prop)
    scf_e = np.array(scf_e)
    exp_ = -scf_e/(kb*T)
    scale_exp_ = exp_/np.amax(exp_)
    avg = np.sum(prop*np.exp(scale_exp_)) / np.sum(np.exp(scale_exp_))
    return avg

In [37]:
dirs_spe = glob.glob('descriptors/mols_crest/*/')

# instantiate mordred descriptors
geom_rad_3d_calc = GeometricalIndex.Diameter3D()
weight_calc = Weight.Weight()
top_rad_2d = TopologicalIndex.Radius()

desc_molcats = {
    'scf_e': {},
    'homo_e': {},
    'lumo_e': {},
    'dip_mom_norm': {},
    'radius_2d': {},
    'mw': {},  
} # the final boltzmann averaged properties

for dir_ in dirs_spe:
    dirs_conf = glob.glob(f'{dir_}/conf-*/')
    print('FRAGMENT :', dir_)
    print('NUM CONFS :', len(dirs_conf))
    f_smi = dir_+'s.smi'
    # read the smiles
    with open(f_smi, 'r') as f:
        smi = f.readlines()[0].strip('\n')
    print('smiles :', smi)
    mol = mfs(smi)
    
    name = dir_.split('/')[-2]
    abbrv = molcats_mapping[name]
    
    print('NAME : ', name)
    print('ABBRV : ', abbrv)
    
    props = {
        'scf_e': [],
        'homo_e': [],
        'lumo_e': [],
        'dip_mom_norm': [],
        'radius_2d': [],
        'mw': [],
    }
    print('DIRS CONF :', dirs_conf)
    
    for conf_dir in dirs_conf:
        
        f_spe = conf_dir+'g16.log'
        if np.logical_and(
            os.path.exists(f_smi), os.path.exists(f_spe)
        ):
        
            # read the dft results
            data = cclib.io.ccread(f_spe)

            mo_energies = data.moenergies
            homo_ix = data.homos[0]
            if len(mo_energies)==1:
                homo_e = data.moenergies[0][homo_ix] # energies in eV
                lumo_e = data.moenergies[0][homo_ix+1] # energy in eV
            elif len(mo_energies)==2: # unresctricted
                homo_e = data.moenergies[1][homo_ix-1] # energies in eV
                lumo_e = data.moenergies[1][homo_ix] # energy in eV

            scf_e = data.scfenergies[0] # energy in eV

            dip_mom_norm = np.linalg.norm(data.moments[1]) # dipole moment in Debye
            
            #-----------------------------------------
            # SKIPPING THE 3d GEOMETRIC STUFF FOR NOW
            #-----------------------------------------
            
#             # get optimized atom coords
#             coords = data.atomcoords[-1]
#             atomnos = data.atomnos
            
#             write_xyz(atomnos, coords, name, filename=f'{conf_dir}dft_conf.xyz')
                
#             molfile = f'{conf_dir}dft_conf.mol'
#             xyz2mol(f'{conf_dir}conformer.xyz')
            
        
#             mol_3d = chem.MolFromMolFile(molfile, removeHs=False)
            
            
#             # compute mordred stuff  (weight is redundant for multiple conformers)
#             diameter_3d = geom_rad_3d_calc(mol_3d)
#            print(diameter_3d)
            
            mw = weight_calc(mol)
            radius_2d = top_rad_2d(mol)

            props['scf_e'].append(scf_e)
            props['homo_e'].append(homo_e)
            props['lumo_e'].append(lumo_e)
            props['dip_mom_norm'].append(dip_mom_norm)
            props['radius_2d'].append(radius_2d)
            props['mw'].append(mw)
            
        
    # perform the boltzmann average over the conformers
    avg_scf_e = get_boltzmann_avg(props['scf_e'], props['scf_e'])
    avg_homo_e = get_boltzmann_avg(props['homo_e'], props['scf_e'])
    avg_lumo_e = get_boltzmann_avg(props['lumo_e'], props['scf_e'])
    avg_dip_mom_norm = get_boltzmann_avg(props['dip_mom_norm'], props['scf_e'])
    avg_radius_2d = get_boltzmann_avg(props['radius_2d'], props['scf_e'])
    avg_mw = get_boltzmann_avg(props['mw'], props['scf_e'])
    
    desc_molcats['scf_e'][abbrv] = avg_scf_e
    desc_molcats['homo_e'][abbrv] = avg_homo_e
    desc_molcats['lumo_e'][abbrv] = avg_lumo_e
    desc_molcats['dip_mom_norm'][abbrv] = avg_dip_mom_norm
    desc_molcats['radius_2d'][abbrv] = avg_radius_2d
    desc_molcats['mw'][abbrv] = avg_mw

    print('-'*50)


FRAGMENT : descriptors/mols_crest/ethylammonium/
NUM CONFS : 1
smiles : CC[NH3+]
NAME :  ethylammonium
ABBRV :  EA
DIRS CONF : ['descriptors/mols_crest/ethylammonium/conf-0/']
--------------------------------------------------
FRAGMENT : descriptors/mols_crest/methylammonium/
NUM CONFS : 1
smiles : C[NH3+]
NAME :  methylammonium
ABBRV :  MA
DIRS CONF : ['descriptors/mols_crest/methylammonium/conf-0/']
--------------------------------------------------
FRAGMENT : descriptors/mols_crest/ammonium/
NUM CONFS : 1
smiles : [NH4+]
NAME :  ammonium
ABBRV :  NH4
DIRS CONF : ['descriptors/mols_crest/ammonium/conf-0/']
--------------------------------------------------
FRAGMENT : descriptors/mols_crest/guanidinium/
NUM CONFS : 1
smiles : C(=[NH2+])(N)N
NAME :  guanidinium
ABBRV :  G
DIRS CONF : ['descriptors/mols_crest/guanidinium/conf-0/']
--------------------------------------------------
FRAGMENT : descriptors/mols_crest/formamidinium/
NUM CONFS : 1
smiles : C(=[NH2+])N
NAME :  formamidinium
A

In [36]:
desc_molcats

{'scf_e': {'EA': -3685.4827558409315,
  'MA': -2616.2996627854645,
  'NH4': -1547.3252089431614,
  'G': -5594.19906880506,
  'FA': -4088.28135385717,
  'tBA': -5823.94438080184,
  'ED': -5172.214043773927,
  'MP': -10410.560772038903,
  'H3S': -11942.468771418577,
  'AA': -5157.813812350571,
  'MS': -11942.468771418577},
 'homo_e': {'EA': -14.8968727456225,
  'MA': -16.9643937817215,
  'NH4': -22.4044938809175,
  'G': -13.262284845668999,
  'FA': -14.001346063627,
  'tBA': -14.192914214379002,
  'ED': -12.536102814649784,
  'MP': -16.7385392858065,
  'H3S': -15.8830133398345,
  'AA': -13.5319496715145,
  'MS': -15.8830133398345},
 'lumo_e': {'EA': -5.480917176770999,
  'MA': -5.7952086740985,
  'NH4': -6.3154903562545,
  'G': -5.185401535128,
  'FA': -6.6188972995620015,
  'tBA': -5.117373072503001,
  'ED': -9.327068507297485,
  'MP': -5.8784755123515,
  'H3S': -6.6556326693795,
  'AA': -6.0937175680969995,
  'MS': -6.6556326693795},
 'dip_mom_norm': {'EA': 3.81036368999076,
  'MA': 2.

In [38]:
# save the descriptors to disc
with open('desc_molcats.pkl', 'wb') as f:
    pickle.dump(desc_molcats, f)

with open('desc_metals.pkl', 'wb') as f:
    pickle.dump(desc_metals, f)

with open('desc_halogens.pkl', 'wb') as f:
    pickle.dump(desc_halogens, f)

In [ ]:
# MOLCAT_DETAILS = {x: get_descriptors(x, desc_molcats) for x in molcats}
# METAL_DETAILS = {x: get_descriptors(x, desc_metals) for x in metals}
# HALOGEN_DETAILS = {x: get_descriptors(x, desc_halogens) for x in halogens}